# Giải thích về hệ số lạm phát phương sai (VIF)

https://etav.github.io/python/vif_factor_python.html

* Colinearity là trạng thái trong đó hai biến có mối tương quan cao và chứa thông tin giống nhau về phương sai trong một tập dữ liệu nhất định. Để phát hiện sự cộng tuyến giữa các biến, chỉ cần tạo một ma trận tương quan và tìm các biến có giá trị tuyệt đối lớn. Trong R, hãy sử dụng corrhàm và trong python, điều này có thể được thực hiện bằng cách sử dụng hàm của numpy corrcoef.

* Mặt khác, đa cộng tuyến khó phát hiện hơn vì nó xuất hiện khi ba biến trở lên, có mối tương quan cao, được đưa vào trong một mô hình. Để làm cho vấn đề tồi tệ nhất, đa cộng tuyến có thể xuất hiện ngay cả khi các cặp biến bị cô lập không cùng tuyến tính.

$$ VIF = 1 / (1 - R^2). $$

Hệ số lạm phát phương sai (VIF) là thước đo tính cộng tuyến giữa các biến dự đoán trong một hồi quy bội. Nó được tính bằng cách lấy tỷ lệ phương sai của tất cả các phiên bản beta của một mô hình nhất định chia cho phương sai của một phiên bản beta nếu nó phù hợp với một mình.

### Các bước thực hiện VIF

1. Chạy hồi quy bội.

2. Tính toán các yếu tố VIF.

3. Kiểm tra các yếu tố cho từng biến dự đoán, nếu VIF nằm trong khoảng từ 5-10, có khả năng xảy ra hiện tượng đa cộng tuyến và bạn nên cân nhắc bỏ biến đó.

In [3]:
#Imports
import pandas as pd
import numpy as np
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

df = pd.read_csv('dataset/loan.csv')
df.dropna()
df = df._get_numeric_data() #drop non-numeric cols

df.head()

C:\Users\Hoai\AppData\Local\Temp\ipykernel_17020\738819735.py:8: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dataset/loan.csv')


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,162.87,24000.0,27.65,0,1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,59.83,30000.0,1.00,0,5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,84.33,12252.0,8.72,0,2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,339.31,49200.0,20.00,0,1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,67.79,80000.0,17.94,0,0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [4]:
df = df[['annual_inc','loan_amnt', 'funded_amnt','annual_inc','dti']].dropna() #subset the dataframe

In [5]:
df

,annual_inc,loan_amnt,funded_amnt,annual_inc,dti
0,24000.0,5000,5000,24000.0,27.65
1,30000.0,2500,2500,30000.0,1.00
2,12252.0,2400,2400,12252.0,8.72
3,49200.0,10000,10000,49200.0,20.00
4,80000.0,3000,3000,80000.0,17.94
...,...,...,...,...,...
39712,110000.0,2500,2500,110000.0,11.33
39713,18000.0,8500,8500,18000.0,6.40
39714,100000.0,5000,5000,100000.0,2.30
39715,200000.0,5000,5000,200000.0,3.72


# Step 1: Run a multiple regression

In [8]:
%%capture
#gather features
features = "+".join(df.columns.difference(["annual_inc"]))

# get y and X dataframes based on this regression: lấy các khung dữ liệu y và X dựa trên hồi quy này. intercept sẽ tự tạo ra do hồi quy
y, X = dmatrices('annual_inc ~' + features, df, return_type='dataframe') 

# nghĩa là hồi annual_inc dựa vào features và df. 

In [9]:
# print(df.columns.difference(["annual_inc"]))

# features = "+".join(df.columns.difference(["annual_inc"]))
# print(features)

In [11]:
X

,Intercept,dti,funded_amnt,loan_amnt
0,1.0,27.65,5000.0,5000.0
1,1.0,1.00,2500.0,2500.0
2,1.0,8.72,2400.0,2400.0
3,1.0,20.00,10000.0,10000.0
4,1.0,17.94,3000.0,3000.0
...,...,...,...,...
39712,1.0,11.33,2500.0,2500.0
39713,1.0,6.40,8500.0,8500.0
39714,1.0,2.30,5000.0,5000.0
39715,1.0,3.72,5000.0,5000.0


# Step 2: Calculate VIF Factors

In [12]:
# For each X, calculate VIF and save in dataframe
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

# Step 3: Inspect VIF Factors

In [14]:
vif.round(1)

,VIF Factor,features
0,6.9,Intercept
1,1.0,dti
2,27.4,funded_amnt
3,27.4,loan_amnt


* Như mong đợi, tổng số tiền được tài trợ cho khoản vay và số tiền của khoản vay có hệ số lạm phát chênh lệch cao vì chúng "giải thích" cùng một phương sai trong tập dữ liệu này. Chúng ta cần loại bỏ một trong các biến này trước khi chuyển sang xây dựng mô hình hoặc mạo hiểm xây dựng một mô hình có tính đa cộng tuyến cao.